In [1]:
pwd

'/mnt/md0/isshamie/Projects/Mito_Trace/workflow/notebooks/clone_af_dendrograms'

In [2]:

# INDIR = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/"
# OUTDIR = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/barcodes/btwnClones_dendro_dt_0.6/"
# DONOR = 0
# dendroThresh=0.6

#clones_meta = '/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/results/clones_summary/feb23_2022/clones_summary.csv'
#clones_indir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/"
#clone_sizes_in_f = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/gff_A2_black/annotation_clones/clone_counts/minCellConds_1/clone_sizes.csv"
#AF_INDIR = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/clones/variants_init/knn/kparam_30/donor0/"

INDIR = "/data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells_constraints_min_both_pct_0.5/clones/variants_prefilterMerge_impute/knn/kparam_30"
OUTDIR = "/data/Mito_Trace/output/pipeline/v04/CHIP_Input_nameFix_april08_2021/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/enriched_barcodes/objs_high_cells_constraints_min_both_pct_0.5/clones/variants_prefilterMerge_impute/knn/kparam_30/barcodes/btwnClones_dendro"
DONOR = 2


# INDIR = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30"
# OUTDIR = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/barcodes/btwnClones_dendro_dt_0.6"
# DONOR = 1
#dendroThresh = 0.6

verbose=False

mt_ref="/data/Mito_Trace/data/processed/genomes/mtMasked/GRCh38_MT_blacklist_A2_2020/chrM.fasta"

In [3]:

import mplh.cluster_help as ch
import mplh.fig_utils as fu

from os.path import join, exists, dirname
from glob import glob
import pickle
import os
import vireoSNP
import numpy as np
from scipy import sparse
from scipy.io import mmread
import matplotlib.pyplot as plt
from scipy.stats import hypergeom
print(vireoSNP.__version__)
import pandas as pd
import seaborn as sns
from vireoSNP import Vireo
np.set_printoptions(formatter={'float': lambda x: format(x, '.5f')})
import scipy
from itertools import combinations
#from src.utils import utils  as ut

fig_utils from mplh
0.5.6


In [4]:
from src.utils.dendro_clust import run_dynamic
import src.utils.variant_utils as vu
import src.utils.dendro_clust as dc



In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
out_f = join(OUTDIR, f"donor{DONOR}")

### Load cells_meta 
### Create clones_meta - get number of cells in a clone and donor

In [7]:
cells_meta = pd.read_csv(join(INDIR, "cells_meta.tsv"), sep="\t", index_col=0)

cells_meta

,orig.ident,nCount_ATAC,nFeature_ATAC,total,duplicate,chimeric,unmapped,lowmapq,mitochondrial,passed_filters,...,seurat_clusters,nCount_RNA,nFeature_RNA,ID,Row.names,lineage,donor,condition,name,cluster_labels
Input_AAACGAATCCCAGTAA-1,Input,2103.0,2103.0,30056.0,2764.0,124.0,63.0,832.0,22385.0,3888.0,...,2.0,2540.0,2215.0,AAACGAATCCCAGTAA-1_Input,Input_AAACGAATCCCAGTAA-1,25,0,Input,0_25,2.0
Input_AAACGAATCGTGAACT-1,Input,5996.0,5996.0,66340.0,10716.0,362.0,165.0,2305.0,39978.0,12814.0,...,4.0,8689.0,5714.0,AAACGAATCGTGAACT-1_Input,Input_AAACGAATCGTGAACT-1,2,0,Input,0_2,4.0
Input_AAACTCGAGGTACACG-1,Input,4901.0,4901.0,49202.0,9164.0,265.0,116.0,1810.0,27219.0,10628.0,...,1.0,6092.0,4446.0,AAACTCGAGGTACACG-1_Input,Input_AAACTCGAGGTACACG-1,4,0,Input,0_4,1.0
Input_AAACTGCAGGATGTAT-1,Input,3421.0,3421.0,34970.0,5801.0,165.0,87.0,972.0,22347.0,5598.0,...,1.0,3701.0,2984.0,AAACTGCAGGATGTAT-1_Input,Input_AAACTGCAGGATGTAT-1,4,0,Input,0_4,1.0
Input_AAACTGCAGTCTCTAG-1,Input,3129.0,3129.0,31794.0,6835.0,210.0,90.0,1208.0,13249.0,10202.0,...,2.0,5219.0,3174.0,AAACTGCAGTCTCTAG-1_Input,Input_AAACTGCAGTCTCTAG-1,2,0,Input,0_2,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Flt3l_ATGTCTTGTGTCGTGC-1,Flt3l,8759.0,8759.0,155564.0,35841.0,999.0,490.0,3409.0,101832.0,12993.0,...,9.0,11376.0,6631.0,ATGTCTTGTGTCGTGC-1_Flt3l,Flt3l_ATGTCTTGTGTCGTGC-1,3,4,Flt3l,4_3,9.0
Flt3l_GTTGGTAAGCGAGAAA-1,Flt3l,7417.0,7417.0,159745.0,37109.0,832.0,641.0,3911.0,106447.0,10805.0,...,9.0,9416.0,5770.0,GTTGGTAAGCGAGAAA-1_Flt3l,Flt3l_GTTGGTAAGCGAGAAA-1,2,4,Flt3l,4_2,9.0
Flt3l_TGGGTGCAGTACCACT-1,Flt3l,8502.0,8502.0,187037.0,52900.0,784.0,1007.0,5934.0,112887.0,13525.0,...,9.0,11193.0,6675.0,TGGGTGCAGTACCACT-1_Flt3l,Flt3l_TGGGTGCAGTACCACT-1,3,4,Flt3l,4_3,9.0
Flt3l_TGGGTGCCAAGTTGCT-1,Flt3l,8723.0,8723.0,189721.0,45559.0,793.0,732.0,4374.0,124486.0,13777.0,...,9.0,10792.0,6534.0,TGGGTGCCAAGTTGCT-1_Flt3l,Flt3l_TGGGTGCCAAGTTGCT-1,3,4,Flt3l,4_3,9.0


In [8]:
cells_meta["lineage"] = cells_meta.apply(lambda x: f'{x["donor"]}_{x["lineage"]}', axis=1)

In [9]:
clones_meta_df = pd.DataFrame(cells_meta.groupby("lineage").size(), columns=["size"]) #, index=["size"])
clones_meta_df

,size
lineage,
0_10,20
0_11,10
0_12,9
0_13,7
0_14,3
0_15,2
0_16,2
0_17,9
0_18,3


In [10]:
clones_meta_df["donor"] = [x.split("_")[0] for x in clones_meta_df.index]
clones_meta_df["donor"] = clones_meta_df["donor"].astype(int)
clones_meta_df

,size,donor
lineage,,
0_10,20,0
0_11,10,0
0_12,9,0
0_13,7,0
0_14,3,0
0_15,2,0
0_16,2,0
0_17,9,0
0_18,3,0


In [11]:
curr_clones = clones_meta_df[clones_meta_df["donor"]==DONOR].copy()
# cells_meta = pd.read_csv(join(INDIR, "cells_meta.tsv"), sep='\t')#.sort_values(["donor", "lineage"])
curr_labels = cells_meta[cells_meta["donor"]==DONOR].copy()

In [12]:
curr_labels

,orig.ident,nCount_ATAC,nFeature_ATAC,total,duplicate,chimeric,unmapped,lowmapq,mitochondrial,passed_filters,...,seurat_clusters,nCount_RNA,nFeature_RNA,ID,Row.names,lineage,donor,condition,name,cluster_labels
Input_TAGACTGCATACAACC-1,Input,4845.0,4845.0,95179.0,9760.0,370.0,164.0,2020.0,74329.0,8536.0,...,3.0,5524.0,4219.0,TAGACTGCATACAACC-1_Input,Input_TAGACTGCATACAACC-1,2_7,2,Input,2_7,3.0
Flt3l_AACAAAGTCAGAGTGG-1,Flt3l,10336.0,10336.0,208290.0,56515.0,1053.0,975.0,8154.0,124057.0,17536.0,...,9.0,13543.0,7599.0,AACAAAGTCAGAGTGG-1_Flt3l,Flt3l_AACAAAGTCAGAGTGG-1,2_10,2,Flt3l,2_10,9.0
Flt3l_GCGAGAAAGTGAAGGA-1,Flt3l,9826.0,9826.0,167384.0,47706.0,1026.0,1904.0,5631.0,93579.0,17538.0,...,9.0,13288.0,7713.0,GCGAGAAAGTGAAGGA-1_Flt3l,Flt3l_GCGAGAAAGTGAAGGA-1,2_7,2,Flt3l,2_7,9.0
Flt3l_GTTCAAGGTTTCGACA-1,Flt3l,8807.0,8807.0,191988.0,53613.0,1198.0,887.0,11228.0,106168.0,18894.0,...,9.0,14727.0,7992.0,GTTCAAGGTTTCGACA-1_Flt3l,Flt3l_GTTCAAGGTTTCGACA-1,2_10,2,Flt3l,2_10,9.0
Flt3l_TAGGTGTGTACTTCAG-1,Flt3l,8122.0,8122.0,226965.0,49607.0,1057.0,938.0,6158.0,155450.0,13755.0,...,9.0,10847.0,6677.0,TAGGTGTGTACTTCAG-1_Flt3l,Flt3l_TAGGTGTGTACTTCAG-1,2_16,2,Flt3l,2_16,9.0
Flt3l_TCAAAGCAGAGCGGTT-1,Flt3l,11617.0,11617.0,259481.0,80115.0,1314.0,1028.0,10333.0,145407.0,21284.0,...,9.0,15886.0,8403.0,TCAAAGCAGAGCGGTT-1_Flt3l,Flt3l_TCAAAGCAGAGCGGTT-1,2_16,2,Flt3l,2_16,9.0


In [13]:
# curr_clones = clones_meta_df[clones_meta_df["donor"]==DONOR].set_index("lineage")
# curr_clones

## Load AF and DP

In [14]:
af_indir = INDIR #join(AF_INDIR) #, "sc_af", f"donor{DONOR}")
AF_df = pd.read_csv(join(af_indir, "af.tsv"), index_col=0, sep="\t").transpose()
DP_df = pd.read_csv(join(af_indir, "dp.tsv"), index_col=0, sep="\t").transpose()


In [15]:
AF_df


,AAACGAATCCCAGTAA-1_Input,AAACGAATCGTGAACT-1_Input,AAACTCGAGGTACACG-1_Input,AAACTGCAGGATGTAT-1_Input,AAACTGCAGTCTCTAG-1_Input,AAACTGCTCCTCCCAA-1_Input,AAAGATGCATGGGACA-1_Input,AAAGGATGTGATGTGG-1_Input,AAAGGGCAGGCCAGTA-1_Input,AAAGGGCCACGATTAC-1_Input,...,TTTGAGGAGTCTCGAT-1_Flt3l,ACGTGGCAGCGATACG-1_Input,CTACTTATCGGGAATG-1_Input,TGGCCTTCAGTATCTG-1_Input,TGGGTTAGTCATTGCA-1_Input,ATGTCTTGTGTCGTGC-1_Flt3l,GTTGGTAAGCGAGAAA-1_Flt3l,TGGGTGCAGTACCACT-1_Flt3l,TGGGTGCCAAGTTGCT-1_Flt3l,TTTGAGGGTTGCAGAG-1_Flt3l
10398G,0.00000,0.016878,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10400T,0.00000,0.017699,0.0,0.0,0.040816,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1040C,0.02381,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.017937,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10559G,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.019417,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1082G,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11204C,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
16519C,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4117C,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
11719A,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.015025,0.004747,0.028714,0.013624,0.008435,0.001738,0.000000,0.012029,0.013947


### Functions to get mean and median

In [16]:
def calc_mean(cells, af):
    print(cells.head())
    return af.loc[:,cells["ID"].values].mean(axis=1)

def calc_median(cells, af):
    #print(cells.head())
    return af.loc[:,cells["ID"].values].median(axis=1)

### Calculate clone-barcode mean

In [19]:
(AF_df == 0).all().sum()

55

In [19]:
mean_af = curr_labels.groupby(["lineage"]).apply(calc_mean, af=AF_df.fillna(0))
mean_af = mean_af.loc[~((mean_af == 0).all(axis=1)),~((mean_af == 0).all(axis=0))]

mean_af

                         orig.ident  nCount_ATAC  nFeature_ATAC  total  \
AAAGGATGTGCTGTGC-1_Flt3l        NaN          NaN            NaN    NaN   
AACAAAGTCTTACGGA-1_Flt3l        NaN          NaN            NaN    NaN   
AACCGATAGGTCGGTA-1_Flt3l        NaN          NaN            NaN    NaN   
AACGTACGTATTCGCA-1_Flt3l        NaN          NaN            NaN    NaN   
AATGGCTAGTGCAACG-1_Flt3l        NaN          NaN            NaN    NaN   

                          duplicate  chimeric  unmapped  lowmapq  \
AAAGGATGTGCTGTGC-1_Flt3l        NaN       NaN       NaN      NaN   
AACAAAGTCTTACGGA-1_Flt3l        NaN       NaN       NaN      NaN   
AACCGATAGGTCGGTA-1_Flt3l        NaN       NaN       NaN      NaN   
AACGTACGTATTCGCA-1_Flt3l        NaN       NaN       NaN      NaN   
AATGGCTAGTGCAACG-1_Flt3l        NaN       NaN       NaN      NaN   

                          mitochondrial  passed_filters  ... seurat_clusters  \
AAAGGATGTGCTGTGC-1_Flt3l            NaN             NaN  ...      

KeyError: "None of [Float64Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n              nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n              nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n              nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n              nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n              nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n              nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n             dtype='float64')] are in the [columns]"

In [ ]:
mean_dp = curr_labels.groupby(["lineage"]).apply(calc_mean, af=np.log2(DP_df+1))
mean_dp = mean_dp.loc[mean_af.index, mean_af.columns]


In [ ]:
anno = mean_af.reset_index()[["lineage"]]

# Remove clones that have all 0
# print('mean_af shape', mean_af.shape)
# mean_af = mean_af.loc[~((mean_af==0).all(axis=1))]
# curr_clones = curr_clones.loc[mean_af.index]
# print('mean_af shape after rm 0', mean_af.shape)


#mean_dp = mean_dp.reset_index(drop=True)
anno_dp = mean_dp.reset_index()[["lineage"]]
na_cells = curr_labels.loc[curr_labels["lineage"].isnull()]
na_mean_af = na_cells.groupby("condition").apply(calc_mean, af=AF_df)

if not (na_mean_af.shape[0] == 0):
    print('na shape', na_mean_af.shape)
    na_anno = pd.concat((anno,pd.DataFrame({"lineage":["None"]*len(na_mean_af.index)})), ignore_index=True)
    na_concat = pd.concat((mean_af.reset_index(drop=True), na_mean_af.reset_index(drop=True)), ignore_index=True)
    na_mean_dp = na_cells.groupby("condition").apply(calc_mean, af=np.log2(DP_df+1))
    na_dp_concat = pd.concat((mean_dp, na_mean_dp.reset_index(drop=True)), ignore_index=True)
else:
    na_anno = anno
    na_concat = mean_af.reset_index(drop=True)
    na_dp_concat = mean_dp
     
mean_af  

In [ ]:
# Get variant types
variants_df = vu.add_ref_to_variants(variants=mean_af.columns.values,
                                     mt_df=vu.load_mt_ref(mt_ref))

variants_df = vu.type_of_variants(variants_df, to_preproc=False)

### Cluster 

In [ ]:
%matplotlib inline
if not (mean_af.shape[0] == 1 or  mean_af.shape[1] == 1 ):
    #den = dc.dendro_cluster(mean_af, g, d_thresh=0.6)
    
    #clusters, link = run_dynamic(mean_af, metric='euclidean', method="average", minClusterSize=1)
    clusters, link = dc.run_dynamic_hyper(np.sqrt(mean_af))
#     is_comp = False
#     for i in ([3,2,1]):
#         try:
#             clusters, link = run_dynamic(mean_af, metric='cosine', method="complete", 
#                                          minClusterSize=i, deepSplit=3)
#             is_comp = True
#             break
#         except IndexError:
#             print(f'didnt work for {i}')
#     print("min cluster size:", i)
    
#     if not is_comp:
#         print('min size 1 and deepsplit 1')
#         clusters, link = run_dynamic(mean_af, metric='cosine', method="complete", 
#                                      minClusterSize=1, deepSplit=1)
        
    curr_clones["den_clust"] = curr_clones.apply(lambda x: f'C{clusters.loc[x.name, "labels"]}', axis=1)

    size_pvals = dc.cluster_stats(curr_clones, out_f= out_f )

    curr_clones["donor"]= [x.split("_")[0] for x in curr_clones.index]
    curr_clones = curr_clones.loc[mean_af.index]

    # with open(out_f + ".dendrogram_pvals.txt", "w") as f:
    #     #for l in size_pvals:
    #     f.write("\n".join(size_pvals))

    curr_clones
else:
    with open(out_f + ".dendrogram_pvals.txt", "w") as f:
        f.write("")

## Plot with dendro clusters

In [ ]:
conditions = cells_meta["condition"].unique()

In [ ]:
if not (mean_af.shape[0] == 1 or  mean_af.shape[1] == 1 ):
    g = ch.plot_cluster(np.sqrt(mean_af),row_meta=curr_clones, to_row_clust=True, to_col_clust=True,
                        col_meta=variants_df[["variant type", "variant change"]], col_clr_schemes="categorical",
                        row_clr_schemes={"size":"sequential","donor":"categorical","den_clust":"categorical"}, 
                        vmax=0.2, vmin=0, row_linkage=link, metric="cosine", method="complete")
    plt.suptitle("Donor sqrt allele-frequency heatmap with clone and condition labels (max AF=0.2)")
    #fu.helper_save(out_f+"withEnrichmentFoldChange.NoCondition.clust.max2.AF.png",to_pdf=False)
    fu.helper_save(out_f+".dendro.NoCondition.max2.AF.png",to_svg=True, to_pdf=False)

    inds = g.dendrogram_row.dendrogram["leaves"]
    cols = g.dendrogram_col.dendrogram["leaves"]

    g1d = ch.plot_cluster(mean_dp.iloc[inds, cols], row_meta=curr_clones, to_row_clust=False, to_col_clust=False,
                          row_clr_schemes={"size":"sequential","donor":"categorical","den_clust":"categorical"}, 
                          col_meta=variants_df[["variant type", "variant change"]], col_clr_schemes="categorical",
                         )
    g1d.ax_cbar.set_title("log2(reads+1)")
    fu.helper_save(out_f+".dendro.NoCondition.DP.png", to_svg=True)#, f=g1d.figure)


    mean_af = mean_af.iloc[inds,cols]
    curr_clones = curr_clones.loc[mean_af.index]

    ch.plot_cluster(np.sqrt(mean_af),row_meta=curr_clones, to_row_clust=False, to_col_clust=False,
                    col_meta=variants_df[["variant type", "variant change"]], col_clr_schemes="categorical",
                    row_clr_schemes={"size":"sequential","donor":"categorical","den_clust":"categorical"}, 
                    vmax=0.5, vmin=0)
    plt.suptitle("Donor sqrt allele-frequency heatmap with clone and condition labels (max AF=0.5)")
    #fu.helper_save(out_f+"withEnrichmentFoldChange.NoCondition.clust.max5.AF.png",to_pdf=False)
    fu.helper_save(out_f+".dendro.NoCondition.max5.AF.png",to_svg=True, to_pdf=False)

else:
    plt.figure()
    fu.helper_save(out_f+".dendro.NoCondition.max5.AF.png",to_svg=True, to_pdf=False)
    fu.helper_save(out_f+".dendro.NoCondition.DP.png", to_svg=True)#, f=g1d.figure)
    fu.helper_save(out_f+".dendro.NoCondition.max2.AF.png",to_svg=True, to_pdf=False)

### Save mean_af and curr_clones

In [ ]:
mean_af.to_csv(out_f+".mean.csv")
curr_clones.to_csv(out_f+".clones_dendro.csv")

In [ ]:
out_f